In [693]:
import numpy as np 
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from copy import deepcopy

from TensorDecisionTreeRegressor import *
#Debugging import
import importlib
var = 'TensorDecisionTreeRegressor'
package = importlib.import_module(var)
for name, value in package.__dict__.items():
    if not name.startswith("__"):
        globals()[name] = value

# Gradient Boosting Regressor Class
from GradientBoosting import *

# Compute the RMSE and RPE
def tensor_RMSE(truth,pred):
    return np.sqrt( np.linalg.norm(truth-pred,ord='fro')  )
def tensor_RPE(truth,pred):
    #res = []
    #for i in range(truth.shape[0]):
    err = tensor_RMSE(truth,pred)**2
    tru = tensor_RMSE(truth,truth*0.)**2
    return (err/tru)**2
    

In [694]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Load face dataset from FaceData.zip corresponding to Section 10 in Lock's 2018 paper https://arxiv.org/abs/1701.01037
# 1 11 15 23 24 31 (32) 33 51 63 | 68 71 90 99 102 117 121 129 135 146
my_seed = 68
np.random.seed(my_seed)
my_task = 'synCP'
my_noise_scalar = 0.01
X_train = np.random.uniform(size=(100,12,6))
X_test = np.random.uniform(size=(100,12,6))
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.cp_tensor import cp_to_tensor
def cp_decompose_reconstruct(tensor, rank):
    """
    Perform CP decomposition and then reconstruct the tensor.
    
    Args:
    tensor (np.ndarray): The input tensor.
    rank (int): The rank for the CP decomposition.

    Returns:
    np.ndarray: Reconstructed tensor using the CP components.
    """
    # Decompose the tensor using CP decomposition
    cp_components = parafac(tensor, rank=rank)
    
    # Reconstruct the tensor from the CP components
    tensor_reconstructed = cp_to_tensor(cp_components)
    
    return tensor_reconstructed

# Set the desired rank for the decomposition
desired_rank = 4

# Perform decomposition and reconstruction
X_train_reconstructed = cp_decompose_reconstruct(X_train, desired_rank)
X_test_reconstructed = cp_decompose_reconstruct(X_test, desired_rank)
print(X_train_reconstructed.shape,X_test_reconstructed.shape)
# Optionally, print or compare the original and reconstructed tensors
print("Original X_train[0]:", X_train[0])
print("Reconstructed X_train[0]:", X_train_reconstructed[0])
print("\nOriginal X_test[0]:", X_test[0])
print("Reconstructed X_test[0]:", X_test_reconstructed[0])
#
y_train = np.zeros(shape=(100,7))
y_test = np.zeros(shape=(100,7))
assert (y_train.shape[1]==y_test.shape[1])
for i in range(y_test.shape[1]):
    N_size = y_test.shape[0]
    y_train[:,i] = (X_train_reconstructed[:,0,1])**2 - X_train_reconstructed[:,0,0] + my_noise_scalar*np.random.uniform(size=(N_size,))
    y_test[:,i] = (X_test_reconstructed[:,0,1])**2 - X_test_reconstructed[:,0,0] + my_noise_scalar*np.random.uniform(size=(N_size,))
        
Lock_X_train = X_train
Lock_X_test = X_test
Lock_y_train = y_train
Lock_y_test = y_test
Lock_X_train = Lock_X_train[:,:,:]
Lock_X_test = Lock_X_test[:,:,:]
Lock_y_train = Lock_y_train[:,:]
Lock_y_test = Lock_y_test[:,:]

np.save(my_task+"_X_train.npy",Lock_X_train)
np.save(my_task+"_X_test.npy",Lock_X_test)
np.save(my_task+"_y_train.npy",Lock_y_test)
np.save(my_task+"_y_test.npy",Lock_y_test)

print(Lock_X_train.shape,Lock_X_test.shape)
print(Lock_y_train.shape,Lock_y_test.shape)
my_rank = 2
my_n_mode = len(Lock_X_train.shape)


(100, 12, 6) (100, 12, 6)
Original X_train[0]: [[0.25937147 0.04336105 0.58203034 0.11251217 0.61077678 0.29638796]
 [0.80725219 0.09483773 0.70010037 0.93289602 0.8422089  0.79995898]
 [0.97282329 0.53608642 0.50723281 0.95483086 0.77569678 0.59055587]
 [0.16887804 0.18339044 0.0317532  0.22962335 0.52991138 0.69191556]
 [0.28996716 0.09009975 0.98563455 0.77317084 0.54878549 0.15826334]
 [0.75044303 0.52469456 0.80580271 0.71656553 0.0158944  0.51318498]
 [0.6730693  0.68454134 0.48631006 0.99791941 0.35928526 0.3664124 ]
 [0.13592195 0.33018249 0.17031529 0.84042189 0.60552385 0.4553598 ]
 [0.70239643 0.99172067 0.63672086 0.01832205 0.13956102 0.64533608]
 [0.00468777 0.07225554 0.56829157 0.66938709 0.84744455 0.25496099]
 [0.73639142 0.99719593 0.02433295 0.99469129 0.10009659 0.79671713]
 [0.07480246 0.2821466  0.0856984  0.90929196 0.72246251 0.78461154]]
Reconstructed X_train[0]: [[0.43010364 0.42221071 0.52962359 0.46038659 0.60219875 0.5819017 ]
 [0.40228663 0.47603992 0.453

In [695]:
# Initialize Gradient Boosting Regressor with a simple decision tree as weak learner
weak_learner = TensorDecisionTreeRegressor(max_depth=3, min_samples_split=4, split_method='variance',split_rank=my_rank,n_mode=my_n_mode)
weak_learner.use_mean_as_threshold = True
weak_learner.sample_rate = 1.0
gradient_boosting_regressor = GradientBoostingRegressor(
    n_estimators=10,
    learning_rate=0.1,
    weak_learner=weak_learner
)
gradient_boosting_regressor.pruning = True
# Fit a single tree model
weak_learner.fit(Lock_X_train, Lock_y_train[:,0])
weak_predictions = weak_learner.predict(Lock_X_test)


# Fit the Gradient Boosting model
gradient_boosting_regressor.fit(Lock_X_train, Lock_y_train[:,0], Lock_X_test, Lock_y_test[:,0])

# Predict
entrywise_CP = gradient_boosting_regressor.predict(Lock_X_test,regression_method='cp')
entrywise_Tucker = gradient_boosting_regressor.predict(Lock_X_test,regression_method='tucker')
#np.save('entrywise_CP_predictions_rank_'+str(my_rank)+'_'+my_task+'.npy',entrywise_CP)
#np.save('entrywise_Tucker_predictions_rank_'+str(my_rank)+'_'+my_task+'.npy',entrywise_Tucker)


# In[21]:


# Display the RMSE
print(entrywise_CP.shape)
print(entrywise_Tucker.shape)
print(np.sqrt(np.mean((Lock_y_test[:,0]-entrywise_CP)**2)))
print(np.sqrt(np.mean((Lock_y_test[:,0]-entrywise_Tucker)**2)))


search size: 72
(7, 5) 0.5318313445775967 0.0018306166565877097
search size: 72
(7, 0) 0.48957700791445374 0.0012271581552447438
search size: 72
(2, 0) 0.4932139391816184 0.0008493221772162979
search size: 72
(6, 2) 0.48694808844322524 0.0009873896817822344
search size: 72
(7, 2) 0.4911806195703359 0.0015862125898909781
search size: 72
(8, 5) 0.5515222053108458 0.0007461506468136739
search size: 72
(1, 2) 0.5024190230751467 0.0015114569132617362
0 0 training MSE === 0.053217270494719415
0 0 testing MSE === 0.09478161039128084
search size: 72
(7, 5) 0.5318313445775967 0.0018306166565877097
search size: 72
(7, 0) 0.48957700791445374 0.0012271581552447438
search size: 72
(2, 0) 0.4932139391816184 0.0008493221772162979
search size: 72
(6, 2) 0.48694808844322524 0.0009873896817822344
search size: 72
(7, 2) 0.4911806195703359 0.0015862125898909781
search size: 72
(8, 5) 0.5515222053108458 0.0007461506468136739
search size: 72
(1, 2) 0.5024190230751467 0.0015114569132617362
pruning a depth= 0

/home/akirahoriguchi/anaconda3/lib/python3.8/site-packages/tensorly/regression/tucker_regression.py:126: RuntimeWarning: divide by zero encountered in double_scalars
  weight_evolution = abs(norm_W[-1] - norm_W[-2]) / norm_W[-1]
/home/akirahoriguchi/anaconda3/lib/python3.8/site-packages/tensorly/regression/tucker_regression.py:126: RuntimeWarning: invalid value encountered in double_scalars
  weight_evolution = abs(norm_W[-1] - norm_W[-2]) / norm_W[-1]


search size: 72
(6, 2) 0.48694808844322524 0.0009873896817822344
search size: 72
(7, 2) 0.4911806195703359 0.0015862125898909781
search size: 72
(8, 5) 0.5515222053108458 0.0007461506468136741
search size: 72
(1, 2) 0.5024190230751467 0.0015114569132617366
pruning a depth= 0
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
0 2 training MSE === 0.03532265143848337
0 2 testing MSE === 0.07321453997248396
search size: 72
(7, 5) 0.5318313445775967 0.00183061665658771
search size: 72
(7, 0) 0.48957700791445374 0.0012271581552447438
search size: 72
(2, 0) 0.4932139391816184 0.0008493221772162981
search size: 72
(6, 2) 0.48694808844322524 0.0009873896817822344
search size: 72
(7, 2) 0.4911806195703359 0.0015862125898909781
search size: 72
(8, 5) 0.5515222053108458 0.0007461506468136739
search size: 72
(1, 2) 0.5024190230751467 0.0015114569132617362
pruning a depth= 0
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
pruni

In [696]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from concurrent.futures import ThreadPoolExecutor
import copy
class GenericTensorRegressor:
    def __init__(self, regressor):
        self.regressor = regressor
        #self.regressor_params = regressor_params
        self.models = []
        
    def fit(self, X_train, y_train):
        # Flatten the tensor if it has more than 2 dimensions
        if y_train.ndim > 2:
            y_train = y_train.reshape(y_train.shape[0], -1)
        
        # Fit a separate model for each column (or flattened column) of y_train
        self.models = []
        for i in range(y_train.shape[1]):
            print(i,',',y_train.shape[1])
            model = copy.deepcopy(self.regressor)
            #if self.regressor is TensorDecisionTreeRegressor:
            #    model.use_mean_as_threshold = True
            model.fit(X_train, y_train[:, i])
            self.models.append(model)

    def fit_parallel(self, X_train, y_train):
        # Flatten the tensor if it has more than 2 dimensions
        if y_train.ndim > 2:
            y_train = y_train.reshape(y_train.shape[0], -1)
        
        # Helper function to fit a model for a specific target column
        def fit_model(column_index):
            print('Fitting model ',column_index)
            model = self.regressor(**self.regressor_params)
            # Example conditional feature, uncomment if needed:
            # if self.regressor is TensorDecisionTreeRegressor:
            #     model.use_mean_as_threshold = True
            model.fit(X_train, y_train[:, column_index])
            return model
        
        # Use ThreadPoolExecutor to parallelize the fitting process
        with ThreadPoolExecutor() as executor:
            self.models = list(executor.map(fit_model, range(y_train.shape[1])))


    def predict(self, X_test, regression_method, output_shape=None):
        # Predict with each model and concatenate the results
        predictions = [model.predict(X_test,regression_method=regression_method) for model in self.models]
        predictions = np.column_stack(predictions)

        # Reshape the predictions to the original tensor shape if specified
        if output_shape is not None and len(output_shape) > 2:
            predictions = predictions.reshape((-1, *output_shape[1:]))
        
        return predictions

# The GenericTensorRegressor can now handle various shapes of y_train automatically.
# It will be used within the TensorOnTensorRegressor class if the output tensor requires it.



In [697]:
# For DecisionTreeRegressor
#tensor_regressor = GenericTensorRegressor(DecisionTreeRegressor)
#tensor_regressor.fit(X_train, y_train)
#predictions = tensor_regressor.predict(X_test)

# For GradientBoostingRegressor
#tensor_regressor = GenericTensorRegressor(GradientBoostingRegressor, n_estimators=50, learning_rate=0.01, loss_function='mse')
#tensor_regressor.fit(X_train, y_train)
#predictions = tensor_regressor.predict(X_test)

# For TensorDecisionTreeRegressor
#weak_learner = TensorDecisionTreeRegressor(max_depth=1, min_samples_split=2, split_method='variance',split_rank=2,n_mode=4)
#weak_learner.use_mean_as_threshold = True
#weak_learner.sample_rate = 0.0001
tensor_regressor = GenericTensorRegressor(gradient_boosting_regressor)
tensor_regressor.fit(Lock_X_train, Lock_y_train)
TTentrywise_GB_CP = tensor_regressor.predict( Lock_X_test,regression_method='cp')
TTentrywise_GB_Tucker = tensor_regressor.predict( Lock_X_test,regression_method='tucker')
np.save('TTentrywise_GB_CP_predictions_rank_'+str(my_rank)+'_'+my_task+'.npy',TTentrywise_GB_CP)
np.save('TTentrywise_GB_Tucker_predictions_rank_'+str(my_rank)+'_'+my_task+'.npy',TTentrywise_GB_Tucker)

0 , 7
0 0 training MSE === 0.0075090671326775126
search size: 72
(7, 5) 0.5318313445775967 0.0018306166565877097
search size: 72
(7, 0) 0.48957700791445374 0.0012271581552447436
search size: 72
(2, 0) 0.4932139391816184 0.0008493221772162979
search size: 72
(6, 2) 0.48694808844322524 0.0009873896817822344
search size: 72
(7, 2) 0.4911806195703359 0.0015862125898909781
search size: 72
(8, 5) 0.5515222053108458 0.0007461506468136739
search size: 72
(1, 2) 0.5024190230751467 0.0015114569132617362
pruning a depth= 0
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
0 1 training MSE === 0.010093226773052246
search size: 72
(7, 5) 0.5318313445775967 0.0018306166565877097
search size: 72
(7, 0) 0.48957700791445374 0.0012271581552447438
search size: 72
(2, 0) 0.4932139391816184 0.0008493221772162979
search size: 72
(6, 2) 0.48694808844322524 0.0009873896817822346
search size: 72
(7, 2) 0.4911806195703359 0.0015862125898909781
sear

In [698]:
print(TTentrywise_GB_CP.shape)
print(TTentrywise_GB_Tucker.shape)
print(tensor_RMSE(Lock_y_test,TTentrywise_GB_CP))
print(tensor_RMSE(Lock_y_test,TTentrywise_GB_Tucker))
    
print(tensor_RPE(Lock_y_test,TTentrywise_GB_CP))
print(tensor_RPE(Lock_y_test,TTentrywise_GB_Tucker))

(100, 7)
(100, 7)
2.410918914120883
2.4309335285484472
0.5087622864837488
0.5258681025277312


In [699]:
class TensorOnTensorRegressor:
    def __init__(self,regressor,rank, decomposition_type='cp'):
        self.decomposition_type = decomposition_type
        self.rank = rank
        self.regressor = regressor
        #self.regressor_params = regressor_params
        self.models = []
        self.decomposition_factors = None
        self.core_tensor = None

    def fit(self, X_train, y_train):
        if self.decomposition_type == 'cp':
            weights, factors = parafac(y_train, rank=self.rank)
            self.decomposition_weights = weights
            self.decomposition_factors = factors
            first_factor = factors[0]

            if first_factor.ndim > 1 and first_factor.shape[1] > 1:
                model = GenericTensorRegressor(regressor=self.regressor)
            else:
                raise RuntimeError('CP Error: Must satisfy first_factor.ndim > 1 and first_factor.shape[1] > 1')
            model.fit(X_train, first_factor)
            self.models.append(model)

        elif self.decomposition_type == 'tucker':
            core, factors = tucker(y_train, rank=[self.rank]*len(y_train.shape))
            self.core_tensor = core
            self.decomposition_factors = factors
            first_factor = factors[0]

            if first_factor.ndim > 1 and first_factor.shape[1] > 1:
                tensor_model = GenericTensorRegressor(regressor=self.regressor)
            else:
                raise RuntimeError('Tucker Error: Must satisfy first_factor.ndim > 1 and first_factor.shape[1] > 1')
            tensor_model.fit(X_train, first_factor)
            self.models.append(tensor_model)

        else:
            raise ValueError("Unsupported decomposition type. Choose 'cp' or 'tucker'.")

    def predict(self, X_test, regression_method):
        predicted_components = [model.predict(X_test,regression_method=regression_method) for model in self.models]
        print(len(predicted_components),'?',predicted_components[0].shape)
        if self.decomposition_type == 'cp':
            # Reconstruct using cp_to_tensor for CP decomposition
            fac_list = [predicted_components[0]]
            for i in range(len(self.decomposition_factors)):
                print(i,'/',len(self.decomposition_factors))
                if i == 0:
                    continue#Skip the training X factor
                fac_list.append(self.decomposition_factors[i])
                print(self.decomposition_factors[i].shape)
            reconstructed = tl.cp_to_tensor((self.decomposition_weights,fac_list))
            return reconstructed
        elif self.decomposition_type == 'tucker':
            # Reconstruct using tucker_to_tensor for Tucker decomposition
            reconstructed = tl.tucker_to_tensor((self.core_tensor, [predicted_components[0]] + self.decomposition_factors[1:]))
            return reconstructed

In [700]:
# Example usage:
tensorOnTensor_reg_CP = TensorOnTensorRegressor(gradient_boosting_regressor,rank=my_rank,decomposition_type='cp')
tensorOnTensor_reg_CP.fit(Lock_X_train, Lock_y_train)
TTlowrank_CP = tensorOnTensor_reg_CP.predict(Lock_X_test,regression_method='cp')
tensorOnTensor_reg_Tucker = TensorOnTensorRegressor(gradient_boosting_regressor,rank=my_rank,decomposition_type='tucker')
tensorOnTensor_reg_Tucker.fit(Lock_X_train, Lock_y_train)
TTlowrank_Tucker = tensorOnTensor_reg_Tucker.predict(Lock_X_test,regression_method='tucker')

0 , 2
0 0 training MSE === 0.21374363442095443
search size: 72
(7, 5) 0.5318313445775967 0.012566295814911704
search size: 72
(7, 0) 0.48957700791445374 0.008516345918520924
search size: 72
(7, 0) 0.1821462010388252 0.005725320834819393
search size: 72
(6, 2) 0.48694808844322524 0.006612550813574428
search size: 72
(7, 2) 0.4911806195703359 0.010744202956728836
search size: 72
(5, 5) 0.6180067494441963 0.004880322859992752
search size: 72
(1, 2) 0.5024190230751467 0.010054450028202154
pruning a depth= 0
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
0 1 training MSE === 0.1938137590244309
search size: 72
(7, 5) 0.5318313445775967 0.012566295814911704
search size: 72
(7, 0) 0.48957700791445374 0.008516345918520922
search size: 72
(7, 0) 0.1821462010388252 0.005725320834819393
search size: 72
(6, 2) 0.48694808844322524 0.006612550813574428
search size: 72
(7, 2) 0.4911806195703359 0.010744202956728836
search size: 72
(5, 

/home/akirahoriguchi/anaconda3/lib/python3.8/site-packages/tensorly/regression/cp_regression.py:116: RuntimeWarning: divide by zero encountered in double_scalars
  weight_evolution = abs(norm_W[-1] - norm_W[-2]) / norm_W[-1]
/home/akirahoriguchi/anaconda3/lib/python3.8/site-packages/tensorly/regression/cp_regression.py:116: RuntimeWarning: invalid value encountered in double_scalars
  weight_evolution = abs(norm_W[-1] - norm_W[-2]) / norm_W[-1]


search size: 72
(1, 5) 0.5435034155752688 1.6890095597457118e-05
pruning a depth= 0
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
0 6 training MSE === 1.4991116952133219e-05
search size: 72
(2, 5) 0.46186723972006954 2.099490642109349e-05
search size: 72
(1, 4) 0.456267242769763 1.682586705262791e-05
search size: 72
(11, 5) 0.5104619199396259 1.1933273670105328e-05
search size: 72
(2, 1) 0.4691277489865972 1.1461270369058809e-05
search size: 72
(10, 0) 0.5161084024544845 1.9137627322185158e-05
search size: 72
(0, 0) 0.537764649258392 1.0720892046121705e-05
search size: 72
(1, 5) 0.5435034155752688 1.6890095597457114e-05
pruning a depth= 0
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
pruning a depth= 1
pruning a depth= 2
pruning a depth= 2
0 7 training MSE === 0.00020187647626646575
search size: 72
(2, 5) 0.46186723972006954 2.099490642109349e-05
search size: 72
(1, 4) 0.456267242769763 1.682586705262791e-05

In [701]:
print(TTlowrank_CP.shape)
print(TTlowrank_Tucker.shape)
print(tensor_RMSE(Lock_y_test,TTlowrank_CP))
print(tensor_RMSE(Lock_y_test,TTlowrank_Tucker))
np.save('TTlowrank_CP_predictions_rank_'+str(my_rank)+'_'+my_task+'.npy',TTlowrank_CP)
np.save('TTlowrank_Tucker_predictions_rank_'+str(my_rank)+'_'+my_task+'.npy',TTlowrank_Tucker)

(100, 7)
(100, 7)
2.3297814663040715
2.072372167138184


In [702]:
Lock_y_test.shape

(100, 7)

In [703]:
import subprocess
#import sys
def run_r_script(rank,task):
    # Define the path to the R script
    r_script_path = './tensorOutput_rrr_synthetic.R'
    
    # Define the command to run the R script with Rscript and pass the rank as an argument
    command = ['Rscript', r_script_path, str(rank),str(task)]
    
    # Run the command
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Print the output of the R script
    print('STDOUT:', result.stdout)
    print('STDERR:', result.stderr)

# Example usage: run the R script with rank 2
run_r_script(my_rank,my_task)

STDOUT: [1] 100  12   6
[1] 100  12   6
[1] 100   7
[1] 100   7
[1] 2
[1] 100   7
[1] 100   7
[1] 0.1842036
[1] 0.1810096
[1] "tensorOutput_rrr_synthetic.R done."

STDERR: 


In [704]:
rrr = np.load("rrr_predictions_rank_"+str(my_rank)+'_'+my_task+'.npy')
print(tensor_RMSE(Lock_y_test,rrr))
print(tensor_RPE(Lock_y_test,rrr))
rrrBayes = np.load("rrrBayes_predictions_rank_"+str(my_rank)+'_'+my_task+'.npy')
print(tensor_RMSE(Lock_y_test,rrrBayes))
print(tensor_RPE(Lock_y_test,rrrBayes))

2.2076180610107654
0.35766813317703283
2.188399391917119
0.3453749505908028


In [705]:
import datetime
def create_and_write_file():
    # Get current datetime
    now = datetime.datetime.now()
    
    # Format datetime as 'DDMMYY_HHMM'
    formatted_date = now.strftime('%d%m%y_%H%M%S')
    
    # Create filename
    filename = f'synthetic_result_{formatted_date}.txt'
    dataname = f'synthetic_datset_{formatted_date}'
    # Lines to write
    lines_to_write = [
        "my_task",
        my_task,
        "my_rank",
        my_rank,
        "my_noise_scalar",
        my_noise_scalar,
        "my_seed",
        my_seed,
        "X_train.shape",
        X_train.shape,
        "X_test.shape",
        X_test.shape,
        "y_train.shape",
        y_train.shape,
        "y_test.shape",
        y_test.shape,
        "---*ALL METRICS BELOW* are evaluated on testing set---",
        "TTentrywise_CP_RMSE",   
        tensor_RMSE(Lock_y_test,TTentrywise_GB_CP),
        "TTentrywise_CP_RPE",   
        tensor_RPE(Lock_y_test,TTentrywise_GB_CP),
        "TTentrywise_Tucker_RMSE",
        tensor_RMSE(Lock_y_test,TTentrywise_GB_Tucker),
        "TTentrywise_Tucker_RPE",   
        tensor_RPE(Lock_y_test,TTentrywise_GB_Tucker),
        "TTlowrank_CP_RMSE",   
        tensor_RMSE(Lock_y_test,TTlowrank_CP),
        "TTlowrank_CP_RPE",   
        tensor_RPE(Lock_y_test,TTlowrank_CP),
        "TTlowrank_Tucker_RMSE",
        tensor_RMSE(Lock_y_test,TTlowrank_Tucker),
        "TTlowrank_Tucker_RPE",   
        tensor_RPE(Lock_y_test,TTlowrank_Tucker),
        "rrr_RMSE",
        tensor_RMSE(Lock_y_test,rrr),
        "rrr_RPE",
        tensor_RPE(Lock_y_test,rrr),
        "rrrBayes_RMSE",
        tensor_RMSE(Lock_y_test,rrrBayes),
        "rrrBayes_RPE",
        tensor_RPE(Lock_y_test,rrrBayes)
    ]
    #print(lines_to_write,sep='\n')
    # Write lines to the file
    with open(filename, 'w') as file:
        for line in lines_to_write:
            file.write(str(line) + '\n')
            print(line)
    
    print(f"File '{filename}' has been created and written successfully.")
    np.save(my_task+"_X_train.npy",X_train)
    np.save(my_task+"_X_test.npy",X_test)
    np.save(my_task+"_y_train.npy",y_train)
    np.save(my_task+"_y_test.npy",y_test)
# Call the function to execute
create_and_write_file()


my_task
synCP
my_rank
2
my_noise_scalar
0.01
my_seed
68
X_train.shape
(100, 12, 6)
X_test.shape
(100, 12, 6)
y_train.shape
(100, 7)
y_test.shape
(100, 7)
---*ALL METRICS BELOW* are evaluated on testing set---
TTentrywise_CP_RMSE
2.410918914120883
TTentrywise_CP_RPE
0.5087622864837488
TTentrywise_Tucker_RMSE
2.4309335285484472
TTentrywise_Tucker_RPE
0.5258681025277312
TTlowrank_CP_RMSE
2.3297814663040715
TTlowrank_CP_RPE
0.4436548549377025
TTlowrank_Tucker_RMSE
2.072372167138184
TTlowrank_Tucker_RPE
0.2777509145629857
rrr_RMSE
2.2076180610107654
rrr_RPE
0.35766813317703283
rrrBayes_RMSE
2.188399391917119
rrrBayes_RPE
0.3453749505908028
File 'synthetic_result_120524_195746.txt' has been created and written successfully.


In [706]:
import os
duration = .5  # seconds
freq = 220  # Hz
os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

32512